# Petits Mementos

* [MarkDown](#MarkDown)
* [Programmation lettrée](#ProgrammationLettree) (avec préprocesseur CPP)
* [Jupyter Multilingue](#JupyterMultilingue)
* [Raccourcis Jupyter](#RaccourcisJupyter)


<a name="MarkDown"></a>
## MarkDown

Structuration :

* \# : titre
* \#\# : sous-titre
* \* : liste
* 1. : liste numérotée

Styles :

* \_italique\_ : _italique_
* \*\*gras\*\*  : **gras**
* \*\*\_gras-italique\_\*\* : **_gras-italique_**
* \~\~barré\~\~  : ~~barré~~

Divers :

* \`\`\`bloc de code\`\`\` : ```bloc de code```
* \[texte\]\(url\) : [caseine](https://caseine.org)
* \!\[texte\]\(url-img\) : ![caseine](https://moodle.caseine.org/pluginfile.php/1/core_admin/logocompact/50x50/1712564507/lait%20caseine.png)


Pour en savoir plus ou aller plus loin :

* menu Help/MarkDown Reference
* pour en savoir plus : https://en.wikipedia.org/wiki/Markdown
* pour aller plus loin : faire du html (ou du latex), par exemple pour affiner un affichage (couleur, tailler, etc.)  
Exemple : \<span style="color: red"\>en rouge\</span\> : <span style="color: red">en rouge</span>

<a name="ProgrammationLettree"></a>
## Magic Python et Programmation ~lettrée

* Magic Python
  * **%%writefile fichier** : en début de cellule, lors de l'exécution de la cellule, un fichier est produit avec le reste de la cellule,    en option \[-a\], ou \[-append\], c'est un ajout au fichier (attention, à n'exécuter l'ajout qu'une fois)
  * **%%sh** : en debut de cellule, lors de l'exécution de la cellule, le reste de la cellule est interprété comme des commandes shell  
  autre forme, en début de ligne, un point d'exclamation, pour exécuter une commande shell, ex : ```!ls```
* Programmation ~lettrée (avec cpp, le préprocesseur C)
  * **#include fichier** 
  * **#define param**  (éventuellement #undef param)
  * **#ifdef param #else #endif** ou #ifndef param #elseif #endif
  * pour appliquer les #include, #define, etc., par exemple le pré-processeur C (avec option -P pour éviter les ajouts pour le C) : **cpp -P fichier**
  * pour définir des param en ligne de commande : **cpp -D param=valeur** 

Pour en savoir plus :
* Magic commands in IPython : https://ipython.readthedocs.io/en/stable/interactive/magics.html
* CPP (le préprocesseur C) : https://gcc.gnu.org/onlinedocs/cpp/

Exemples (en C) :

* exemple général : #include et #définition de variable en ligne de commande

In [3]:
%%writefile declaration.cx
char nom[100];

Overwriting declaration.cx


In [4]:
%%writefile saisie.cx
printf("Nom ?\n");
scanf("%99s",&nom);

Overwriting saisie.cx


In [1]:
%%writefile prog.cx
int main() {
#include "declaration.cx"
#include "saisie.cx"
printf(MSGBONJOUR);
printf("\n%s",nom);
return 0;}

Writing prog.cx


In [51]:
!cpp -P -D MSGBONJOUR="\"Bien le bonjour\"" prog.cx

int main() {
char nom[100];
printf("Nom ?\n");
scanf("%99s",&nom);
printf("Bien le bonjour");
printf("\n%s",nom);
return 0;}


* Exemple sur les valeurs intiales d'une variable (valeur par défaut et redéfinition à l'exécution)

In [2]:
%%writefile prog.cx
#ifndef DEFNOMINIT
#define DEFNOMINIT "Alan"
#endif
int main() {
char nom[20]=DEFNOMINIT;
printf("Bonjour %s\n",nom);
return 0;}

Overwriting prog.cx


In [8]:
!cpp -P prog.cx

int main() {
char nom[20]="Alan";
printf("Bonjour %s\n",nom);
return 0;}


In [7]:
!cpp -P -D DEFNOMINIT="\"John\"" prog.cx 

int main() {
char nom[20]="John";
printf("Bonjour %s\n",nom);
return 0;}


<a name="JupyterMultilingue"></a>
## Jupyter Multilingue

Jupyter, via Python, peut être utilisé avec tous les langages disponibles sur le serveur. Entre autres :

* ARM 
* Bash
* C 
* C++
* Erlang
* Haskell
* Java 
* OCaml
* Prolog 
* Python 
* R

_([exemples ci-après](#ExempleMultilingue))_

Pour les entrées-sorties, quelques pistes de solution (avec le C comme exemple)

* Solution 1, avec des entrées prévues et passées sur la ligne de commande

In [39]:
%%writefile bonjour.c
#include <stdio.h>

int main() {
    char prenom[100], nom[100];
    printf("Prenom ?\n");
    scanf("%99s",&prenom);
    printf("Nom ?\n");
    scanf("%99s",&nom);
    printf("\nBonjour %s, %s.",prenom,nom);
    return 0;}

Overwriting bonjour.c


In [38]:
!gcc bonjour.c
!echo "Alan\nTuring" | ./a.out #selon la config. de echo, parfois utiliser echo -e

Prenom ?
Nom ?

Bonjour Alan, Turing.

* Solution 2, avec saisie interactive (sous réserve d'avoir des affichages immédiats (sinon, ils arrivent un peu tard) et un programme d'interaction en Python (qui sait gérer les entrées))

In [4]:
import os
import time

def lanceur(prog,nb=4,tps=20):  
    # prog : le chemin de l'exécutable
    # nb : le nombre d'interactions prévues
    # tps : le temps maximum prévu pour l'exécution (interactions comprises)
    os.system("if [ -p monPipeEnEntree ]; then rm monPipeEnEntree; fi")
    os.system("mkfifo monPipeEnEntree")
    os.system("sleep "+str(tps)+" > monPipeEnEntree &")         # important : ouvrir et garder le pipe ouvert pour un temps assez grand
    os.system("(sleep "+str(tps+1)+" ; rm monPipeEnEntree) &")
    os.system(prog+" < monPipeEnEntree &")                      # important : lancer le programme (prévoir que les sorties soient visibles, par ex. sur la sortie erreur)
    for i in range(nb):
        time.sleep(0.5)
        x=input()                                            # important : faire une saisie (cela peut servir)
        os.system("echo \""+x+"\" > monPipeEnEntree")               # important : envoyer la saisie sur le programme (si cela peut servir)
    os.system("if [ -p monPipeEnEntree ]; then rm monPipeEnEntree; fi")   

In [1]:
%%writefile bonjour.c
#include <stdio.h>

int main() {
    char prenom[100], nom[100];
    setbuf(stdout,0);           //à noter, imposer des affichages immédiats 
    printf("Prenom ?\n");
    scanf("%99s",&prenom);
    printf("Nom ?\n");
    scanf("%99s",&nom);
    printf("\nBonjour %s, %s.",prenom,nom);
    return 0;}

Writing bonjour.c


In [2]:
!gcc bonjour.c

In [5]:
lanceur("./a.out",2,60)

Prenom ?


 Alan


Nom ?


 Turing



Bonjour Alan, Turing.

&nbsp;  
   
   
<a name="ExempleMultilingue"></a>
**(suite)**  
\[...\], des exemples de programmes dans plusieurs langues.

# ARM

In [15]:
%%writefile es.s
  .global EcrChaine
  .text
@ EcrChaine :
@    ecriture de la chaine dont l'adresse est dans r1
EcrChaine:
   mov ip, sp
   stmfd sp!, {r0, r1, r2, r3, fp, ip, lr, pc}
   sub     fp, ip, #4
   ldr r0, LD_fe_chaine
   bl printf
   ldmea fp, {r0, r1, r2, r3, fp, sp, pc}
LD_fe_chaine: .word fe_chaine
  .data
fe_chaine: .asciz "%s\n"


Overwriting es.s


In [14]:
%%writefile bonjour.s
  .global main
  .data
msg: .asciz "Bonjour"
  .text
main:
    push {lr}
    ldr R1,LD_msg
    bl EcrChaine
    pop {lr}
    bx lr
    
LD_msg: .word msg

Overwriting bonjour.s


In [20]:
%%sh
rm -f bonjour.e
arm-linux-gnueabi-gcc -static -c es.s
arm-linux-gnueabi-gcc -static -c bonjour.s
arm-linux-gnueabi-gcc -static bonjour.o es.o -o bonjour.e
qemu-arm  bonjour.e
echo "(version ARM)"

Bonjour
(version ARM)


# Bash

In [21]:
%%writefile bonjour.sh
#!/bin/bash

echo "Bonjour"

Writing bonjour.sh


In [24]:
%%sh
chmod a+x bonjour.sh
./bonjour.sh

Bonjour


# C

In [5]:
%%writefile bonjour.c
#include <stdio.h>

int main() {
    printf("Bonjour");
    return 0;}


Writing bonjour.c


In [6]:
%%sh
gcc bonjour.c -o bonjour
./bonjour

Bonjour

# C++

In [9]:
%%writefile bonjour.cpp
#include <iostream>
using namespace std;

int main() {
  cout << "Bonjour \n";
  return 0;}

Writing bonjour.cpp


In [12]:
%%sh
g++ bonjour.cpp -o bonjour
./bonjour

Bonjour 


# Erlang

In [7]:
%%writefile bonjour.erl
-module(bonjour).
-compile([export_all,nowarn_export_all]).

main([]) -> 
  io:format("Bonjour",[]).
  

Writing bonjour.erl


In [8]:
%%sh
escript bonjour.erl

Bonjour

# Haskell 

In [1]:
%%writefile prog.hs
main = do 
    putStrLn "Bonjour "

Writing prog.hs


In [3]:
!runhugs prog.hs

Bonjour 


# Java

In [7]:
%%writefile bonjour.java
import java.util.Scanner;
public class Prog {
    public static void main(String[] args) {
        System.out.println("Bonjour ");}}

Overwriting bonjour.java


In [8]:
%%sh
java bonjour.java

Bonjour 


# OCaml

In [3]:
%%writefile bonjour.ml
let _ = 
    print_endline ("Bonjour ");;

Overwriting bonjour.ml


In [4]:
%%sh
ocaml bonjour.ml

Bonjour 


# Prolog

In [9]:
%%writefile bonjour.pl
main :- 
  writeln('Bonjour').

:- main.

Writing bonjour.pl


In [10]:
%%sh
swipl -q -s bonjour.pl

Bonjour



In [ ]:
%%sh
swipl -g halt -s bonjour.pl

Bonjour



# Python

In [5]:
print("Bonjour")

Bonjour


# R

In [3]:
%%writefile bonjour.r
cat("bonjour")

Writing bonjour.r


In [ ]:
%%sh
Rscript bonjour.r

<a name="RaccourcisJupyter"></a>
# Raccourcis Jupyter

* Mode édition vs mode Commande :  
  * en mode édition : **escape** passe en mode commande ; 
  * en mode commande : **enter** passe en mode édition
* en mode commande : 
  * **Y** passe la cellule en mode Code, 
  * **M** passe la cellule en mode Markdows, 
  * **R** passe la cellule en mode Raw
* en mode commande : 
  * **A** ajouter une cellule au dessus de la cellule courante (above), 
  * **B** ajouter une cellule sous la cellule courante (below), 
  * **X** détruit la cellule courante (attention !)
* Split vs Merge 
  * en mode édition, **Ctrl+Shift+Moins** (essayer le - du pavé numérique ?) : sépare un cellule en 2
  * en mode commande, **Shift+M** : fusionne la cellule courante et la cellule dessous
* En savoir plus : Menu Settings/Advanced/Keyboard shortcuts

## Fin